# State Tomography of Steane

In [1]:
from qiskit import QuantumCircuit
import numpy as np
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError
from qiskit.visualization import plot_histogram 
import typing
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

from qiskit_aer import Aer
from qiskit import transpile
from qiskit_aer.primitives import Sampler
# from qiskit.primitives.backend_sampler import BackendSampler
from qiskit.quantum_info import Statevector
from qiskit.circuit import Instruction
from matplotlib import pyplot as plt
from qiskit.quantum_info import state_fidelity

In [2]:
import steane_encoding_module as sem

In [3]:
import steane_functions as sf

## Theta state

In [4]:
theta = np.arctan(np.sqrt((np.sqrt(5) - 1) / 2))
amp_0_theta = np.cos(theta/2)
amp_1_theta = np.sin(theta/2)

get the steane encoding circuit

In [5]:
circ_ideal = sf.steane_enc_7(amp_0_theta, amp_1_theta, is_clbits=False)
circ_noise = sf.steane_enc_7(amp_0_theta, amp_1_theta, is_clbits=False)

## Z measurements

In [22]:
def z_basis(qc, num_qubits):
    qc.save_density_matrix(qubits=range(num_qubits), label="rho")
    return qc

## Y measurements

In [23]:
def y_basis(qc, num_qubits):
    for qb in range(num_qubits):
        qc.sdg(qb)
        qc.h(qb)
    qc.save_density_matrix(qubits=range(num_qubits), label="rho")
    return qc

## X measurements

In [24]:
def x_basis(qc, num_qubits):
    for qb in range(num_qubits):
        qc.h(qb)
    qc.save_density_matrix(qubits=range(num_qubits), label="rho")
    return qc

# State tomography

In [25]:
from qiskit.quantum_info import DensityMatrix

In [26]:
def qst_create_id_block(num: int, is_clbits: bool):

    if (is_clbits == 1):
        id_block = QuantumCircuit(num, num, name = "id_block")
    else:
        id_block = QuantumCircuit(num, name = "id_block")
    

    for i in range(0, num):
        id_block.id(i)

    id_block_instr = id_block.to_instruction()
    return id_block_instr

In [27]:
def qst_prepare_state(qc_prep: QuantumCircuit, encode_block: Instruction, log_block: Instruction, num: int, is_clbits: bool):

        qc_prep.append(qst_create_id_block(num, is_clbits), range(0,num))
        qc_prep.append(encode_block, range(0,num))
        qc_prep.append(qst_create_id_block(num, is_clbits), range(0,num))
        qc_prep.append(log_block,range(0,num))
        qc_prep.draw('mpl')

## Z basis

In [ ]:
qc = QuantumCircuit(7)
z_intr_ideal = circ_ideal.to_instruction()
num = 7
shots = 10
print(z_intr_ideal._num_clbits)
qst_prepare_state(qc, z_intr_ideal, qst_create_id_block(7,is_clbits=False), 7, is_clbits=False)

z_dm = DensityMatrix.from_instruction(qc)
print(z_dm)



z_basis(qc, 7)

# print(qc)


simulator = AerSimulator(method='density_matrix')
#simulator = Aer.get_backend('density_matrix')

steane_t = transpile(qc, simulator)

# #print(steane_t)
# st_dm = DensityMatrix.from_instruction(steane_t)

# print(st_dm)

job = simulator.run(steane_t, shots=1)

result = job.result().data()

# dm = result.get('density_matrix')

print(np.array(result['rho']))




# print(st_dm)
# print(dm)
# counts = result.get_counts()
# state_ideal = result.get_statevector()
# sv_ideal = Statevector(state_ideal)

0
DensityMatrix([[0.10112712+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.04913446+0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               ...,
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.04913446+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.02387288+0.j]],
              dims=(2, 2, 2, 2, 2, 2, 2))
[[ 1.01127124e-01-2.22908554e-35j -5.58779419e-34+6.19225045e-18j
  -5.50411146e-34+6.19225045e-18j ... -1.13696171e-33+3.00861803e

In [39]:
qc = QuantumCircuit(7)
z_intr_noise = circ_noise.to_instruction()
err_list_size = 6
error_list = np.logspace((-3), (-2), err_list_size)
factor = 10

qst_prepare_state(qc, z_intr_noise, qst_create_id_block(7,is_clbits=False), 7, is_clbits=False)

noise_list = []
for error in error_list:
        # readout_err = ReadoutError([[0.98, 0.02],  # P(measured 0 | actual 0), P(1 | 0)
        #                             [0.02, 0.98]]) # P(0 | 1), P(1 | 1)

        noise_model = NoiseModel()

        error2 = error*factor

        noise_model.add_all_qubit_quantum_error(depolarizing_error(error,1), ['id'])
        noise_model.add_all_qubit_quantum_error(depolarizing_error(error,1), ['h'])
        noise_model.add_all_qubit_quantum_error(depolarizing_error(error2,2),['cz'])
        noise_model.add_all_qubit_quantum_error(depolarizing_error(error2,2),['cx'])

        readout_err = ReadoutError([[1-error2, error2],  # P(measured 0 | actual 0), P(1 | 0)
                                [error2, 1-error2]]) # P(0 | 1), P(1 | 1)
        
        for i in range(0,7):
            noise_model.add_readout_error(readout_err, [i])
        
        noise_list.append(noise_model)

z_dm = DensityMatrix.from_instruction(qc)
print(z_dm)


z_basis(qc, 7)


simulator = AerSimulator(method='density_matrix', noise_model=noise_list[5])
#simulator = Aer.get_backend('density_matrix')

steane_t = transpile(qc, simulator)

# #print(steane_t)
# st_dm = DensityMatrix.from_instruction(steane_t)


# # print(z_dm)
# print(st_dm)



steane_t = transpile(qc, simulator)
job = simulator.run(steane_t, shots=1)
result = job.result().data()



# dm = result.get('density_matrix')


print(np.array(result['rho']))


DensityMatrix([[0.10112712+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.04913446+0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               ...,
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.04913446+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.02387288+0.j]],
              dims=(2, 2, 2, 2, 2, 2, 2))
[[ 4.40584525e-02+2.49083622e-19j -1.83715717e-19-7.12268189e-36j
  -3.26916451e-19-6.39965907e-36j ... -1.75216346e-19+2.68083900e-3

In [19]:
noise_list[1]

<NoiseModel on ['h', 'cz', 'cx', 'id', 'measure']>

## X basis

In [40]:
qc = QuantumCircuit(7)
x_intr_ideal = circ_ideal.to_instruction()
num = 7
shots = 10

qst_prepare_state(qc, x_intr_ideal, qst_create_id_block(7,is_clbits=False), 7, is_clbits=False)

x_dm = DensityMatrix.from_instruction(qc)
print(x_dm)


x_basis(qc, 7)


simulator = AerSimulator(method='density_matrix')
steane_t = transpile(qc, simulator)
job = simulator.run(steane_t, shots=1)
result = job.result().data()


print(np.array(result['rho']))
x_dm = np.array(result['rho'])


DensityMatrix([[0.10112712+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.04913446+0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               ...,
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.04913446+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.02387288+0.j]],
              dims=(2, 2, 2, 2, 2, 2, 2))
[[0.11163446+0.j 0.        +0.j 0.        +0.j ... 0.        +0.j
  0.        +0.j 0.03862712+0.j]
 [0.        +0.j 0.        +0.j 0.

In [44]:
qc = QuantumCircuit(7)
x_intr_noise = circ_noise.to_instruction()
err_list_size = 6
error_list = np.logspace((-3), (-2), err_list_size)
factor = 10

qst_prepare_state(qc, x_intr_noise, qst_create_id_block(7,is_clbits=False), 7, is_clbits=False)

noise_list = []
for error in error_list:
        # readout_err = ReadoutError([[0.98, 0.02],  # P(measured 0 | actual 0), P(1 | 0)
        #                             [0.02, 0.98]]) # P(0 | 1), P(1 | 1)

        noise_model = NoiseModel()

        error2 = error*factor

        noise_model.add_all_qubit_quantum_error(depolarizing_error(error,1), ['id'])
        noise_model.add_all_qubit_quantum_error(depolarizing_error(error,1), ['h'])
        noise_model.add_all_qubit_quantum_error(depolarizing_error(error2,2),['cz'])
        noise_model.add_all_qubit_quantum_error(depolarizing_error(error2,2),['cx'])

        readout_err = ReadoutError([[1-error2, error2],  # P(measured 0 | actual 0), P(1 | 0)
                                [error2, 1-error2]]) # P(0 | 1), P(1 | 1)
        
        for i in range(0,7):
            noise_model.add_readout_error(readout_err, [i])
        
        noise_list.append(noise_model)

x_dm = DensityMatrix.from_instruction(qc)
print(x_dm)

x_basis(qc, 7)


simulator = AerSimulator(method='density_matrix', noise_model=noise_list[5])
steane_t = transpile(qc, simulator)


job = simulator.run(steane_t, shots=1)
result = job.result().data()



print(np.array(result['rho']))
x_dm = np.array(result['rho'])


DensityMatrix([[0.10112712+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.04913446+0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               ...,
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.04913446+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.02387288+0.j]],
              dims=(2, 2, 2, 2, 2, 2, 2))
[[0.05307356+0.j 0.        +0.j 0.        +0.j ... 0.        +0.j
  0.        +0.j 0.01141225+0.j]
 [0.        +0.j 0.00802576+0.j 0.

## Y basis

In [43]:
qc = QuantumCircuit(7)
y_intr_ideal = circ_ideal.to_instruction()
num = 7
shots = 10

qst_prepare_state(qc, y_intr_ideal, qst_create_id_block(7,is_clbits=False), 7, is_clbits=False)

y_dm = DensityMatrix.from_instruction(qc)
print(y_dm)


y_basis(qc, 7)


simulator = AerSimulator(method='density_matrix')
steane_t = transpile(qc, simulator)
job = simulator.run(steane_t, shots=1)
result = job.result().data()


print(np.array(result['rho']))
x_dm = np.array(result['rho'])

DensityMatrix([[0.10112712+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.04913446+0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               ...,
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.04913446+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.02387288+0.j]],
              dims=(2, 2, 2, 2, 2, 2, 2))
[[ 6.25000000e-02+1.30061031e-34j  1.91351062e-18+1.79056418e-34j
   1.91351062e-18+1.61944795e-34j ... -1.18261460e-18-1.50430901e-1

In [45]:
qc = QuantumCircuit(7)
y_intr_noise = circ_noise.to_instruction()
err_list_size = 6
error_list = np.logspace((-3), (-2), err_list_size)
factor = 10

qst_prepare_state(qc, y_intr_noise, qst_create_id_block(7,is_clbits=False), 7, is_clbits=False)

noise_list = []
for error in error_list:
        # readout_err = ReadoutError([[0.98, 0.02],  # P(measured 0 | actual 0), P(1 | 0)
        #                             [0.02, 0.98]]) # P(0 | 1), P(1 | 1)

        noise_model = NoiseModel()

        error2 = error*factor

        noise_model.add_all_qubit_quantum_error(depolarizing_error(error,1), ['id'])
        noise_model.add_all_qubit_quantum_error(depolarizing_error(error,1), ['h'])
        noise_model.add_all_qubit_quantum_error(depolarizing_error(error2,2),['cz'])
        noise_model.add_all_qubit_quantum_error(depolarizing_error(error2,2),['cx'])

        readout_err = ReadoutError([[1-error2, error2],  # P(measured 0 | actual 0), P(1 | 0)
                                [error2, 1-error2]]) # P(0 | 1), P(1 | 1)
        
        for i in range(0,7):
            noise_model.add_readout_error(readout_err, [i])
        
        noise_list.append(noise_model)

y_dm = DensityMatrix.from_instruction(qc)
print(y_dm)

y_basis(qc, 7)


simulator = AerSimulator(method='density_matrix', noise_model=noise_list[5])
steane_t = transpile(qc, simulator)


job = simulator.run(steane_t, shots=1)
result = job.result().data()



print(np.array(result['rho']))
y_dm = np.array(result['rho'])


DensityMatrix([[0.10112712+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.04913446+0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               ...,
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.        +0.j],
               [0.04913446+0.j, 0.        +0.j, 0.        +0.j, ...,
                0.        +0.j, 0.        +0.j, 0.02387288+0.j]],
              dims=(2, 2, 2, 2, 2, 2, 2))
[[ 2.68300982e-02+0.00000000e+00j  1.42182984e-18+1.21972744e-19j
   1.28194860e-18+2.50721752e-19j ... -6.99385030e-19-1.11952639e-1